<a href="https://colab.research.google.com/github/thunil/jupyter-notebooks/blob/main/SoL_karman2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reducing Numerical Errors with Deep Learning

Next, we'll target numerical errors that arise in the discretization of a continuous PDE $\mathcal P^*$, i.e. when we formulate $\mathcal P$. This approach will demonstrate that, despite the lack of closed-form descriptions, discretization errors often are functions with regular and repeating structures and, thus, can be learned by a neural network. Once the network is trained, it can be evaluated locally to improve the solution of a PDE-solver, i.e., to reduce its numerical error. The resulting method is a hybrid one: it will always run (a coarse) PDE solver, and then improve if at runtime with corrections inferred by an NN.

Pretty much all numerical methods contain some form of iterative process. That can be repeated updates over time for explicit solvers,or within a single update step for implicit solvers. Below we'll target iterations over time, an example for the second case could be found [here](https://github.com/tum-pbs/CG-Solver-in-the-Loop).

In the context of reducing errors, it's crucial to have a _differentiable physics solver_, so that the learning process can take the reaction of the solver into account. This interaction is not possible with supervised learning or PINN training. Even small inference errors of a supervised model can accumulate over time, and lead to a data distribution that differs from the distribution of the pre-computed data. This distribution shift can lead to sub-optimal results, or even cause blow-ups of the solver.

In order to learn the error function, we'll consider two different discretizations of the same PDE $\mathcal P^*$: 
a _reference_ version, which we assume to be accurate, with a discretized version 
$\mathcal P_r$, and solutions $\mathbf r \in \mathscr R$, where $\mathscr R$ denotes the manifold of solution of $\mathcal P_r$.
In parallel to this, we have a less accurate approximation of the same PDE, which we'll refer to as the _source_ version, as this will be the solver that our NN should later on interact with. Analogously,
we have $\mathcal P_s$ with solutions $\mathbf s \in \mathscr S$.
And after training, we'll have a _hybrid_ solver that uses $\mathcal P_s$ in conjunction with a trained network to obtain improved solutions, i.e., solutions that are closer to the ones produced by $\mathcal P_r$.

```{figure} resources/diffphys-sol-manifolds.jpeg
---
height: 280px
name: diffphys-sol-manifolds
---
Visual overview of coarse and reference manifolds
```



explain coarse/ref setup...
Let's assume $\mathcal{P}$ advances a solution by a time step $\Delta t$, and let's denote $n$ consecutive steps by a superscript:
$
\newcommand{\pde}{\mathcal{P}}
\newcommand{\pdec}{\pde_{s}}
\newcommand{\vc}[1]{\mathbf{s}_{#1}} 
\newcommand{\vr}[1]{\mathbf{r}_{#1}} 
\newcommand{\vcN}{\vs}          
\newcommand{\project}{\mathcal{T}}   
\vc{t+n} = \pdec(\pdec(\cdots \pdec( \project \vr{t}  )\cdots)) = \pdec^n ( \project \vr{t} ) .
$
Here we assume a mapping operator $\mathcal{T}$ exists that transfers a reference solution to the source manifold. This could, e.g., be a simple downsampling operation.
Especially for longer sequences, i.e. larger $n$, the source state 
$\newcommand{\vc}[1]{\mathbf{s}_{#1}} \vc{t+n}$
will deviate from a corresponding reference state
$\newcommand{\vr}[1]{\mathbf{r}_{#1}} \vr{t+n}$. 
This is what we will address with an NN in the following.

We'll use an $L^2$-norm in the following to quantify the deviations, i.e., 
$
\newcommand{\loss}{\mathcal{L}} 
\newcommand{\corr}{\mathcal{C}} 
\newcommand{\vc}[1]{\mathbf{s}_{#1}} 
\newcommand{\vr}[1]{\mathbf{r}_{#1}} 
\loss (\vc{t},\project \vr{t})=\Vert\vc{t}-\project \vr{t}\Vert_2$. 
Our learning goal is to train at a correction operator $\corr ( \vc{} )$ such that 
a solution to which the correction is applied has a lower error than the original unmodified (source) solution: 
$\loss ( \pdec( \corr (\project \vr{t_0}) ) , \project \vr{t_1}) < \loss ( \pdec( \project \vr{t_0} ), \project \vr{t_1})$. 

The correction function 
$\newcommand{\vcN}{\mathbf{s}} \newcommand{\corr}{\mathcal{C}} \corr (\vcN | \theta)$ 
is represented as a deep neural network with weights $\theta$
and receives the state $\vcN$ to infer an additive correction field with the same dimension.
To distinguish the original states $\vcN$ from the corrected ones, we'll denote the latter with an added tilde
$\newcommand{\vctN}{\tilde{\mathbf{s}}} \vctN$.
The overall learning goal now becomes

$
\text{argmin}_\theta | 
( \pdec \corr )^n ( \project \vr{t} )
- \project \vr{t}|^2
$

A crucial bit here that's easy to overlook is that the correction depends on the modified states, i.e.
it is a function of
$\newcommand{\vctN}{\tilde{\mathbf{s}}} \vctN$, so we have 
$\newcommand{\vctN}{\tilde{\mathbf{s}}} \newcommand{\corr}{\mathcal{C}} \corr (\vctN | \theta)$.
These states actually evolve over time when training. They don't exist beforehand.

**TL;DR**:
We'll train a network $\mathcal{C}$ to reduce the numerical errors of a simulator with a more accurate reference. Here it's crucial to have the _source_ solver realized as a differential physics operator, such that it can give gradients for an improved training of $\mathcal{C}$.

\\

---

First, let's download the prepared data set (for details on generation & loading cf. https://github.com/tum-pbs/Solver-in-the-Loop), and let's get the data handling out of the way, so that we can focus on the _interesting_ parts...

In [ ]:
import os, sys, logging, argparse, pickle, glob, random, distutils.dir_util

if not os.path.isfile('data-karman2d-train.pickle'):
  import urllib.request
  url="https://ge.in.tum.de/download/2020-solver-in-the-loop/sol-karman-2d-data.pickle"
  print("Downloading training data (73MB), this can take a moment the first time...")
  urllib.request.urlretrieve(url, 'data-karman2d-train.pickle')

with open('data-karman2d-train.pickle', 'rb') as f: dataPreloaded = pickle.load(f)
print("Loaded data, {} training sims".format(len(dataPreloaded)) )


Loaded data, 6 training sims


Also let's get installing / importing all the necessary libraries out of the way. And while we're at it, we can set the random seed - obviously, 42 is the ultimate choice here 🙂

In [ ]:
!pip install --upgrade --quiet phiflow
%tensorflow_version 1.x

from phi.tf.flow import *
import phi.tf.util

import tensorflow as tf
from tensorflow import keras

random.seed(42)
np.random.seed(42)
tf.compat.v1.set_random_seed(42)


     |████████████████████████████████| 2.7MB 8.8MB/s 
TensorFlow 1.x selected.
Could not load resample cuda libraries: CUDA binaries not found at /usr/local/lib/python3.6/dist-packages/phi/tf/cuda/build/resample.so. Run "python setup.py cuda" to compile them





/usr/local/lib/python3.6/dist-packages/phi/viz/display.py:80: UserWarning: GUI is disabled because of missing dependencies: No module named 'dash_core_components'. To install all dependencies, run $ pip install phiflow[gui]
  warnings.warn('GUI is disabled because of missing dependencies: %s. To install all dependencies, run $ pip install phiflow[gui]' % import_error)
/usr/local/lib/python3.6/dist-packages/phi/tf/flow.py:15: UserWarning: TensorFlow-CUDA solver is not available. To compile it, download phiflow sources and run
$ python setup.py tf_cuda
before reinstalling phiflow.
  warnings.warn("TensorFlow-CUDA solver is not available. To compile it, download phiflow sources and run\n$ python setup.py tf_cuda\nbefore reinstalling phiflow.")


Now we can set up the _source_ simulation $\newcommand{\pdec}{\pde_{s}} \pdec$. 
Note that we won't deal with 
$\newcommand{\pder}{\pde_{r}} \pder$
below: the downsampled reference data is contained in the training data set. It was generated with a four times finer discretization. Below we're focusing on the interaction of the source solver and the NN. 

This code block and the next ones will define lots of functions, that will be used later on for training.

The `KarmanFlow` solver below simulates a relatively standard wake flow case with a spherical obstacle in a rectangular domain, and an explicit viscosity solve to obtain different Reynolds numbers. This is the geometry of the setup:

```{figure} resources/diffphys-sol-domain.png
---
height: 200px
name: diffphys-sol-domain
---
Domain setup for the wake flow case.
```

The solver applies inflow boundary conditions for the y-velocity with a pre-multiplied mask (`velBCy,velBCyMask`), and then calls `super().step()` to run the _regular_ phiflow fluid solving step.


In [ ]:
class KarmanFlow(IncompressibleFlow):
    def __init__(self, pressure_solver=None, make_input_divfree=False, make_output_divfree=True):
        IncompressibleFlow.__init__(self, pressure_solver, make_input_divfree, make_output_divfree)

        self.infl = Inflow(box[5:10, 25:75])
        self.obst = Obstacle(Sphere([50, 50], 10))

    def step(self, fluid, re, res, velBCy, velBCyMask, dt=1.0, gravity=Gravity()):
        # apply viscosity
        alpha = 1.0/math.reshape(re, [fluid._batch_size, 1, 1, 1]) * dt * res * res

        cy = diffuse(CenteredGrid(fluid.velocity.data[0].data), alpha)
        cx = diffuse(CenteredGrid(fluid.velocity.data[1].data), alpha)

        # apply velocity BCs, only for y velocity for now. note: content of velBCy should be pre-multiplied
        cy = cy*(1.0 - velBCyMask) + velBCy

        fluid = fluid.copied_with(velocity=StaggeredGrid([cy.data, cx.data], fluid.velocity.box))

        return super().step(fluid=fluid, dt=dt, obstacles=[self.obst], gravity=gravity, density_effects=[self.infl], velocity_effects=())


Network arch: fully convolutional
input: 2 fields with x,y velociy, plus Reynolds number as constant channel
output: 2 component velocity field

simple model, using keras for simplicity.

In [ ]:
def model_small(tensor_in):
    return keras.Sequential([
        keras.layers.Input(tensor=tensor_in),
        keras.layers.Conv2D(filters=32, kernel_size=5, padding='same', activation=tf.nn.relu),
        keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation=tf.nn.relu),
        keras.layers.Conv2D(filters=2,  kernel_size=5, padding='same', activation=None),  # u, v
    ])


and for flexibility (and larger-scale tests), let's define a _proper_ ResNet with a few more layers

In [ ]:
def model_medium(tensor_in):
    l_input = keras.layers.Input(tensor=tensor_in)
    block_0 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(l_input)
    block_0 = keras.layers.LeakyReLU()(block_0)

    l_conv1 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(block_0)
    l_conv1 = keras.layers.LeakyReLU()(l_conv1)
    l_conv2 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(l_conv1)
    l_skip1 = keras.layers.add([block_0, l_conv2])
    block_1 = keras.layers.LeakyReLU()(l_skip1)

    l_conv3 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(block_1)
    l_conv3 = keras.layers.LeakyReLU()(l_conv3)
    l_conv4 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(l_conv3)
    l_skip2 = keras.layers.add([block_1, l_conv4])
    block_2 = keras.layers.LeakyReLU()(l_skip2)

    l_conv5 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(block_2)
    l_conv5 = keras.layers.LeakyReLU()(l_conv5)
    l_conv6 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(l_conv5)
    l_skip3 = keras.layers.add([block_2, l_conv6])
    block_3 = keras.layers.LeakyReLU()(l_skip3)

    l_conv7 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(block_3)
    l_conv7 = keras.layers.LeakyReLU()(l_conv7)
    l_conv8 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(l_conv7)
    l_skip4 = keras.layers.add([block_3, l_conv8])
    block_4 = keras.layers.LeakyReLU()(l_skip4)

    l_conv9 = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(block_4)
    l_conv9 = keras.layers.LeakyReLU()(l_conv9)
    l_convA = keras.layers.Conv2D(filters=32, kernel_size=5, padding='same')(l_conv9)
    l_skip5 = keras.layers.add([block_4, l_convA])
    block_5 = keras.layers.LeakyReLU()(l_skip5)

    l_output = keras.layers.Conv2D(filters=2,  kernel_size=5, padding='same')(block_5)
    return keras.models.Model(inputs=l_input, outputs=l_output)


pretty important functions: transform simulation state into input tensor for network

and then, after network evaluation, transform output tensor into phiflow staggered velocity grid (warning: two _centered_ grids with different sizes, leave work to `unstack_staggered_tensor` function in `StaggeredGrid` constructor).

In [ ]:
def to_feature(fluidstate, ext_const_channel):
    # drop the unused edges of the staggered velocity grid making its dim same to the centered grid's
    with tf.name_scope('to_feature') as scope:
        return math.concat(
            [
                fluidstate.velocity.staggered_tensor()[:, :-1:, :-1:, 0:2],
                tf.constant(shape=fluidstate.density.data.shape, value=1.0)*math.reshape(value=ext_const_channel, shape=[fluidstate._batch_size, 1, 1, 1]),
            ],
            axis=-1
        )

def to_staggered(tensor_cen, box):
    with tf.name_scope('to_staggered') as scope:
        return StaggeredGrid(math.pad(tensor_cen, ((0,0), (0,1), (0,1), (0,0))), box=box)


we also need some data handling

load all "ground truth" reference data, already downsampled!

we actually have a lot of data: multiple simulations, with many time steps, each with different fields

data format: ['sim', frame, field (dens/vel)] where each field is [batch-size, y-size, x-size, channels]

In [ ]:
class Dataset():
    def __init__(self, data_preloaded, num_frames, num_sims=None, batch_size=1):
        self.epoch         = None
        self.epochIdx      = 0
        self.batch         = None
        self.batchIdx      = 0
        self.step          = None
        self.stepIdx       = 0

        self.dataPreloaded = data_preloaded
        self.batchSize     = batch_size

        self.numSims       = num_sims
        self.numBatches    = num_sims//batch_size
        self.numFrames     = num_frames
        self.numSteps      = num_frames

        #with open('/Users/thuerey/temp/sol-karman-2d-data.pickle', 'rb') as f: self.dataPreloaded = pickle.load(f)
        
        self.dataSims = ['karman-fdt-hires-set/sim_%06d'%i for i in range(num_sims) ]
        self.dataFrms = [ np.arange(num_frames) for _ in self.dataSims ]  

        # constant additional per-sim channel: Reynolds numbers from data generation
        ReNrs = [160000.0, 320000.0, 640000.0,  1280000.0,  2560000.0,  5120000.0]
        self.extConstChannelPerSim = { self.dataSims[i]:[ReNrs[i]] for i in range(num_sims) }

        #print(format(self.dataPreloaded[self.dataSims[0]][0][0].shape )) # debugging example: check shape of a single density field

        # the data has the following shape ['sim', frame, field (dens/vel)] where each field is [batch-size, y-size, x-size, channels]
        self.resolution = self.dataPreloaded[self.dataSims[0]][0][0].shape[1:3]  

        # compute data statistics for normalization
        self.dataStats = {
            'std': (
                np.std(np.concatenate([np.absolute(self.dataPreloaded[asim][i][0].reshape(-1)) for asim in self.dataSims for i in range(num_frames)], axis=-1)),  # density
                (
                    np.std(np.concatenate([np.absolute(self.dataPreloaded[asim][i][1][...,0].reshape(-1)) for asim in self.dataSims for i in range(num_frames)])),  # vel[0]
                    np.std(np.concatenate([np.absolute(self.dataPreloaded[asim][i][1][...,1].reshape(-1)) for asim in self.dataSims for i in range(num_frames)])),  # vel[1]
                )
            ),
            'ext.std': [ np.std([np.absolute(self.extConstChannelPerSim[asim][0]) for asim in self.dataSims]) ] # Re
        }

        print("Data stats: "+format(self.dataStats))

    # re-shuffle data for next epoch
    def newEpoch(self, exclude_tail=0, shuffle_data=True):
        self.numSteps = self.numFrames - exclude_tail
        simSteps = [ (asim, self.dataFrms[i][0:(len(self.dataFrms[i])-exclude_tail)]) for i,asim in enumerate(self.dataSims) ]
        sim_step_pair = []
        for i,_ in enumerate(simSteps):
            sim_step_pair += [ (i, astep) for astep in simSteps[i][1] ]  # (sim_idx, step) ...

        if shuffle_data: random.shuffle(sim_step_pair)
        self.epoch = [ list(sim_step_pair[i*self.numSteps:(i+1)*self.numSteps]) for i in range(self.batchSize*self.numBatches) ]
        self.epochIdx += 1
        self.batchIdx = 0
        self.stepIdx = 0

    def nextBatch(self):  
        self.batchIdx += self.batchSize
        self.stepIdx = 0

    def nextStep(self):
        self.stepIdx += 1


add function ...

In [ ]:
    # for class Dataset():

    # get one mini batch of data: [marker density, velocity, Reynolds number] all from ground truth
    def getData(self, consecutive_frames, with_skip=1):
        marker_dens = [
            math.concat([
                self.dataPreloaded[
                    self.dataSims[self.epoch[self.batchIdx+i][self.stepIdx][0]]  # sim_key
                ][
                    self.epoch[self.batchIdx+i][self.stepIdx][1]+j*with_skip  # steps
                ][0]
                for i in range(self.batchSize)
            ], axis=0) for j in range(consecutive_frames+1)
        ]
        velocity = [
            math.concat([
                self.dataPreloaded[
                    self.dataSims[self.epoch[self.batchIdx+i][self.stepIdx][0]]  # sim_key
                ][
                    self.epoch[self.batchIdx+i][self.stepIdx][1]+j*with_skip  # steps
                ][1]
                for i in range(self.batchSize)
            ], axis=0) for j in range(consecutive_frames+1)
        ]
        ext = [
            self.extConstChannelPerSim[
                self.dataSims[self.epoch[self.batchIdx+i][self.stepIdx][0]]
            ][0] for i in range(self.batchSize)
        ]
        return [marker_dens, velocity, ext]


next

In [ ]:
output_dir = "./"  # TODO create? , replaced params['train'] and params['tf']
nsims = 6
batch_size = 3
simsteps = 500

dataset = Dataset( data_preloaded=dataPreloaded, num_frames=simsteps, num_sims=nsims, batch_size=batch_size )
#dataset.newEpoch()
#print(format(getData(dataset,1)))
#print(format(dataset.getData(1)))


Data stats: {'std': (2.2194703, (0.32598782, 0.1820292)), 'ext.std': [1732512.6262166172]}


init

setup additional global variables and simulation

very important: `msteps`

In [ ]:
# one of the most crucial! how many simulation steps to look into the future while training
msteps = 4

# this is the actual resolution in terms of cells
source_res = list(dataset.resolution)
# this is only a virtual size, in terms of abstract units for the bounding box of the domain (in practice it's important for conversions or when rescaling to physical units)
sim_len = 100.

source     =   Fluid(Domain(resolution=source_res, box=box[0:sim_len*2, 0:sim_len], boundaries=OPEN), buoyancy_factor=0, batch_size=batch_size)
reference = [ Fluid(Domain(resolution=source_res, box=box[0:sim_len*2, 0:sim_len], boundaries=OPEN), buoyancy_factor=0, batch_size=batch_size) for _ in range(msteps) ]

# velocity BC
vn = np.zeros(source.velocity.data[0].data.shape)  # st.velocity.data[0] is considered as the velocity field in y axis!
vn[..., 0:2, 0:vn.shape[2]-1, 0] = 1.0
vn[..., 0:vn.shape[1], 0:1,   0] = 1.0
vn[..., 0:vn.shape[1], -1:,   0] = 1.0
velBCy = vn
velBCyMask = np.copy(vn)  # warning, mask needs to be binary, 0..1, this only works if vel is also 1

lr_in =   tf.placeholder(tf.float32, shape=[])  # learning rate
Re_in =   tf.placeholder(tf.float32, shape=[batch_size])  # Reynolds numbers

source_in =   phi.tf.util.placeholder_like(source)
reference_in = [ phi.tf.util.placeholder_like(source) for _ in range(msteps) ]

model = model_small(to_feature(source_in, Re_in)) # use small model for testing
#model = model_medium(to_feature(source_in, Re_in)) # optionally switch to larger model
model.summary() 



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (3, 64, 32, 32)           2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (3, 64, 32, 64)           51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (3, 64, 32, 2)            3202      
Total params: 56,898
Trainable params: 56,898
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: placeholder_like may not respect the batch dimension. For State objects, use placeholder(state.shape) instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: placeholder_like may not respect the batch dimension. For State objects, use placeholder(state.shape) instead.


finally, some sim steps

In [ ]:
prediction, correction = [], []
for i in range(msteps):
    prediction += [
        KarmanFlow().step(
            source_in if i==0 else prediction[-1],
            re=Re_in,
            res=source_res[-1],  # reference resolution is size in x direction
            velBCy=velBCy,
            velBCyMask=velBCyMask
        )
    ]

    correction += [
        to_staggered(
            model(
                to_feature(prediction[-1], Re_in)/[
                    *(dataset.dataStats['std'][1]),  # velocity
                    dataset.dataStats['ext.std'][0]  # Re
                ]
            ) * (dataset.dataStats['std'][1]),
            box=source.velocity.box
        )
    ]

    prediction[-1] = prediction[-1].copied_with(velocity=prediction[-1].velocity + correction[-1])


loss calc

In [ ]:
loss_steps = [
    tf.nn.l2_loss(
        (reference_in[i].velocity.staggered_tensor() - prediction[i].velocity.staggered_tensor())
        /dataset.dataStats['std'][1]
    )
    for i in range(msteps)
]
loss = tf.reduce_sum(loss_steps)/msteps

# skip ???
# for i,a_step_loss in enumerate(loss_steps): tf.compat.v1.summary.scalar(name='loss_step{:02d}'.format(i), tensor=a_step_loss)
# tf.compat.v1.summary.scalar(name='total_loss', tensor=loss)
# tf.compat.v1.summary.scalar(name='lr', tensor=lr_in)


setup training

In [ ]:
lr = 1e-4
adapt_lr = True
resume = 0 # load model existing model?
epochs = 10

opt = tf.compat.v1.train.AdamOptimizer(learning_rate=lr_in)
train_step = opt.minimize(loss)

tf_session = tf.Session() 
scene = Scene.create(output_dir, count=batch_size, mkdir=False, copy_calling_script=False)
sess = Session(scene, session=tf_session)
tf.compat.v1.keras.backend.set_session(tf_session)

sess.initialize_variables()

# optional, load existing model...
if resume>0: 
    ld_model = keras.models.load_model(output_dir+'/model_epoch{:04d}.h5'.format(resume))
    model.set_weights(ld_model.get_weights())


another small helper

In [ ]:

def lr_schedule(epoch, current_lr):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 10, 15, 20, 22 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = current_lr
    if   epoch == 23: lr *= 0.5
    elif epoch == 21: lr *= 1e-1
    elif epoch == 16: lr *= 1e-1
    elif epoch == 11: lr *= 1e-1
    return lr


finally, start training!

In [ ]:
current_lr = lr
steps = 0
for j in range(epochs):  # training
    dataset.newEpoch(exclude_tail=msteps)
    if j<resume:
        print('resume: skipping {} epoch'.format(j+1))
        steps += dataset.numSteps*dataset.numBatches
        continue

    current_lr = lr_schedule(j, current_lr) if adapt_lr else lr
    for ib in range(dataset.numBatches):   
        for i in range(dataset.numSteps): 
            batch = getData(dataset, consecutive_frames=msteps) # should be dataset.getData
            re_nr = batch[2] # Reynolds numbers
            source = source.copied_with(density=batch[0][0], velocity=batch[1][0])
            reference = [ reference[k].copied_with(density=batch[0][k+1], velocity=batch[1][k+1]) for k in range(msteps) ]

            my_feed_dict = { source_in: source, Re_in: re_nr, lr_in: current_lr }
            my_feed_dict.update(zip(reference_in, reference))
            _, l2 = sess.run([train_step, loss], my_feed_dict)
            steps += 1

            print('epoch {:03d}/{:03d}, batch {:03d}/{:03d}, step {:04d}/{:04d}: loss={}'.format( j+1, epochs, ib+1, dataset.numBatches, i+1, dataset.numSteps, l2 ))
            dataset.nextStep()

        dataset.nextBatch()

    if j%10==9: model.save(output_dir+'/model_epoch{:04d}.h5'.format(j+1))

#tf_writer_tr.close()
model.save(output_dir+'/model.h5')


Streaming output truncated to the last 5000 lines.
epoch 005/010, batch 002/002, step 0457/0496: loss=3.40366792678833
epoch 005/010, batch 002/002, step 0458/0496: loss=3.3733911514282227
epoch 005/010, batch 002/002, step 0459/0496: loss=2.1802964210510254
epoch 005/010, batch 002/002, step 0460/0496: loss=2.727764844894409
epoch 005/010, batch 002/002, step 0461/0496: loss=3.267439126968384
epoch 005/010, batch 002/002, step 0462/0496: loss=2.7953271865844727
epoch 005/010, batch 002/002, step 0463/0496: loss=3.1806530952453613
epoch 005/010, batch 002/002, step 0464/0496: loss=4.665388107299805
epoch 005/010, batch 002/002, step 0465/0496: loss=3.5460848808288574
epoch 005/010, batch 002/002, step 0466/0496: loss=2.616530418395996
epoch 005/010, batch 002/002, step 0467/0496: loss=2.7210044860839844
epoch 005/010, batch 002/002, step 0468/0496: loss=3.0125303268432617
epoch 005/010, batch 002/002, step 0469/0496: loss=2.7860662937164307
epoch 005/010, batch 002/002, step 0470/0496:

loss should go down from ca. 1000 to around 1

next test eval?